# MAT281 - Tarea 2

**Indicaciones**:

* En los **Ejercicio 1-8** puedes utilizar tanto `matplotlib` como `altair` según te parezca más conveniente o cómodo, en ambos casos cada gráfico debe tener elementos mínimos como:
    - Título
    - Nombre de los ejes, leyendas, etc. en formato _amigable_/_humano_, por ejemplo, si la columna del dataframe en cuestión tiene por nombre `casos_confirmados` se espera que el eje del gráfico tenga por nombre `Casos confirmados`.
    - Colores adecuados al tipo de datos.
    - Un tamaño adecuado para ver con facilidad en una pantalla con resolución HD o FullHD.
    - Cada vez que no se cumplan alguna de estos requerimientos se descontará __1 punto__ de la nota final.

* Para el **Ejercicio 9** es obligación utilizar `altair`.
* Cada ejercicio debe estar acompañado con una celda con comentarios o análisis que puedas desprender de los gráficos.

In [30]:
import numpy as np
import pandas as pd   #Una vez me tiro un bug acá de que pandas no posee core,(creo que así era) el bug es propio de la versión de pandas (hay que reiniciar el binder y se arregla.)
import matplotlib.pyplot as plt
import altair as alt
import ipywidgets as widgets

from datetime import date
from ipywidgets import interactive, interact

pd.set_option('display.max_columns', 999)
alt.data_transformers.enable('data_server')
alt.themes.enable('opaque')

%matplotlib inline

**COVID-19 en Chile** 

En esta tarea exploraremos los datos de Covid-19 en Chile a profundidad. Las siguientes celdas cargarán los datos a utilizar en tu sesión. Es importante que leas la documentación de cada conjunto de datos para comprender las columnas.

In [2]:
start_date = pd.to_datetime("2020-04-13")

In [3]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto6
confirmados = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto6/bulk/data.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
    .dropna()
    .astype({"casos_confirmados": np.float, "tasa": np.float})
)

confirmados.head()

,poblacion,casos_confirmados,fecha,region_id,region,provincia_id,provincia,comuna_id,comuna,tasa
0,247552.0,270.0,2020-04-27,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,109.067994
1,247552.0,9138.0,2020-10-12,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,3691.345657
2,247552.0,596.0,2020-05-29,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,240.757497
3,247552.0,6131.0,2020-08-17,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,2476.651370
4,247552.0,353.0,2020-05-11,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,142.596303


In [4]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto19
activos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .loc[lambda x: x["codigo_comuna"].notnull()]
    .melt(id_vars=["region", "codigo_region", "comuna", "codigo_comuna", "poblacion"], var_name="fecha", value_name="casos_activos")
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
)

activos.head()

,region,codigo_region,comuna,codigo_comuna,poblacion,fecha,casos_activos
0,Arica y Parinacota,15,Arica,15101.0,247552.0,2020-04-13,88.0
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,2020-04-13,0.0
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,2020-04-13,0.0
3,Arica y Parinacota,15,Putre,15201.0,2515.0,2020-04-13,0.0
4,Tarapaca,1,Alto Hospicio,1107.0,129999.0,2020-04-13,8.0


In [5]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto14
fallecidos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["region"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos

,region,fecha,fallecidos
374,Arica y Parinacota,2020-04-13,1.0
375,Tarapacá,2020-04-13,0.0
376,Antofagasta,2020-04-13,1.0
377,Atacama,2020-04-13,0.0
378,Coquimbo,2020-04-13,0.0
...,...,...,...
4330,Los Ríos,2020-12-01,55.0
4331,Los Lagos,2020-12-01,203.0
4332,Aysén,2020-12-01,14.0
4333,Magallanes,2020-12-01,195.0


In [6]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto10
fallecidos_etareo = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["grupo_de_edad"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
        grupo_de_edad=lambda x: x["grupo_de_edad"].str.replace("<=39", "0-39")
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos_etareo

,grupo_de_edad,fecha,fallecidos
28,0-39,2020-04-13,3
29,40-49,2020-04-13,2
30,50-59,2020-04-13,3
31,60-69,2020-04-13,12
32,70-79,2020-04-13,29
...,...,...,...
1654,50-59,2020-12-01,1547
1655,60-69,2020-12-01,3256
1656,70-79,2020-12-01,4337
1657,80-89,2020-12-01,3987


## Ejercicio 1

(10 puntos)

Mostrar cantidad de fallecidos a la fecha por cada grupo etáreo.

In [7]:
c=0
fallecidos_etareofin= fallecidos_etareo.drop(fallecidos_etareo[fallecidos_etareo['fecha']<fallecidos_etareo['fecha'].max()].index).reset_index() #df con solo registros de la ultima fecha
for i in fallecidos_etareo['grupo_de_edad'].unique() :  #for para imprimir 
    print(f"Los fallecidos en el grupo ({i}) es: {fallecidos_etareofin.loc[c , 'fallecidos']} \n")
    c=c+1



Los fallecidos en el grupo (0-39) es: 350 

Los fallecidos en el grupo (40-49) es: 519 

Los fallecidos en el grupo (50-59) es: 1547 

Los fallecidos en el grupo (60-69) es: 3256 

Los fallecidos en el grupo (70-79) es: 4337 

Los fallecidos en el grupo (80-89) es: 3987 

Los fallecidos en el grupo (>=90) es: 1434 



**Comentarios:** Con esto podemos ver que la población de mayor riesgo son los adultos de tercera edad.


## Ejercicio 2

(10 puntos)

¿Qué tan variable es la población de las comunas de Chile? Considera utilizar un gráfico que resuma de buena forma la información sin agregar la variable de región o provincia.

In [8]:
#alt.Chart(confirmados).mark_line().encode(
#    x='fecha',
#    y='poblacion',
#    color='comuna'
#)
#Podemos ver que la población es constante en el tiempo para todas las comunas
#Así que este gráfico es más adecuado para ver que tan variable es las cantidad de población para las distintas comunas de Chile 
poblacionxcomuna=alt.Chart(confirmados).mark_bar().encode(
    x=alt.X('comuna' , axis=alt.Axis(title='Comuna')),        
    y=alt.Y('poblacion' , axis=alt.Axis(title='Población')),
).properties(
    title="Población de las comunas de Chile",
    )

poblacionxcomuna 

alt.Chart(...)

**Comentarios:** Podemos ver por simple inspección que hay la población de las distintas comunas de Chile es bastante variable.

## Ejercicio 3

(10 puntos)

Mostrar evolución y comparación de los fallecimientos entre distintos grupos etáreos, pero que al mismo tiempo sea fácil identificar la cantidad de fallecidos total en cada fecha.

In [9]:
fallecidos_x_grupo_etareo = []  #lista de df (vacia)
graf_fallecidos_x_etareo= []   #lista de graficos (vacia)
for i in fallecidos_etareo['grupo_de_edad'].unique():  #crea distintos df con solo datos del grupo etareo i y guarda cada df en la lista
    fallecidos_x_grupo_etareo.append(fallecidos_etareo.drop(fallecidos_etareo[fallecidos_etareo['grupo_de_edad']!=i].index))


for i in fallecidos_x_grupo_etareo: #agrega a la segunda lista vacia graficos con los df anteriores
    graf_fallecidos_x_etareo.append(alt.Chart(i).mark_line().encode(
        x=alt.X('fecha' , axis=alt.Axis(title='Fecha')),        
        y=alt.Y('fallecidos' , axis=alt.Axis(title='Fallecidos')),  
        color= 'grupo_de_edad'
    ).properties(
    title="Evolución de fallecimientos entre distintos grupos etarios",
    width=800,
    height=400
    )
                                    )
Todito=alt.Chart(fallecidos_etareo.groupby("fecha").agg({"fallecidos" : "sum"}).reset_index()).mark_line(color="pink").encode(
    x=alt.X('fecha' , axis=alt.Axis(title='Fecha')),        
    y=alt.Y('fallecidos' , axis=alt.Axis(title='Fallecidos')),
    )
   
    
    
#sum(graf_fallecidos_x_etareo)
(graf_fallecidos_x_etareo[0] + graf_fallecidos_x_etareo[1] + graf_fallecidos_x_etareo[2] +  graf_fallecidos_x_etareo[3] 
 + graf_fallecidos_x_etareo[4] + graf_fallecidos_x_etareo[5] + graf_fallecidos_x_etareo[6]) + Todito

alt.LayerChart(...)

**Comentarios:** (Rosado: Cantidad total de fallecidos en cada día). Podemos ver que la mayor cantidad de fallecidos son de grupos etarios correspondientes a la tercera edad.

## Ejercicio 4

(10 puntos)

Mostrar en tres gráficos la evolución de casos confirmados, evolución de fallecimientos y evolución de casos activos.

In [10]:
#confirmados.groupby('fecha').agg({"casos_confirmados": "sum"})
Graf_confirmados=alt.Chart(confirmados).mark_line().encode(   
        x=alt.X('fecha' , axis=alt.Axis(title='Fecha')),   
        y=alt.X('sum(casos_confirmados)' , axis=alt.Axis(title='Casos confirmados')),
).properties(
    title='Evolución de casos confirmados',
    width=1000,
    height=400
)

Graf_activos=alt.Chart(activos).mark_line().encode(
        x=alt.X('fecha' , axis=alt.Axis(title='Fecha')), 
        y=alt.X('sum(casos_activos)' , axis=alt.Axis(title='Casos activos')),
).properties(
    title='Evolución de casos activos',
    width=1000,
    height=400
)

Graf_fallecidos=alt.Chart(fallecidos.query("region == 'Total'")).mark_line().encode(
        x=alt.X('fecha' , axis=alt.Axis(title='Fecha')), 
        y=alt.X('fallecidos' , axis=alt.Axis(title='Fallecidos')),
).properties(
    title='Evolución de fallecimientos',
    width=1000,
    height=400
)
Graf_confirmados & Graf_fallecidos & Graf_activos  

alt.VConcatChart(...)

**Comentarios:** RESPONDE AQUÍ

## Ejercicio 5

(10 puntos)

Comparar la tasa de incidencia entre las regiones a lo largo del tiempo.

In [11]:
alt.Chart(confirmados.groupby(['region','fecha']).agg({'tasa':'sum'}).reset_index()).mark_line().encode(
        x=alt.X('fecha' , axis=alt.Axis(title='Fecha')),         
        y=alt.Y('tasa' , axis=alt.Axis(title='Tasa de incidencia')),   #Ojo ver si hay que calcular algo
        color= 'region'  #color para implementar y diferenciar las regiones
    
    ).properties(
    title= "Evolución de tasa de incidencia por región",
    width=900,
    height=550
)

alt.Chart(...)

**Comentarios:** Es claro que la mayor evolución en la tasa de incidencia se encuentra en la región Metropolitana dde Santiago.

## Ejercicio 6

(10 puntos)

¿Hay alguna conclusión que puedas obtener rápidamente al graficar un _scatter plot_ con los casos confirmados y tasa de incidencia de cada comuna para los días 13 de abril y 6 de noviembre del 2020? Además, colorea cada punto según la región a la que pertenece y considera si es útil en el gráfico que el tamaño sea proporcional a la población.

In [32]:
abril13=alt.Chart(confirmados.drop(confirmados[confirmados['fecha']!='2020-04-13'].index)).mark_circle().encode(
    x=alt.X('casos_confirmados' , axis=alt.Axis(title='Casos confirmados')),
    y=alt.Y('tasa' , axis=alt.Axis(title='Tasa de incidencia')),
    color='region',
    size='poblacion'
).properties(
    title= "Gráfico de dispersión 13/04/2020",
    width=500,
    height=350
)



nov06=alt.Chart(confirmados.drop(confirmados[confirmados['fecha']!='2020-11-06'].index)).mark_circle(size=45).encode(
    x=alt.X('casos_confirmados' , axis=alt.Axis(title='casos confirmados')),
    y=alt.Y('tasa' , axis=alt.Axis(title='Tasa de incidencia')),
    color='region',
    size='poblacion'
).properties(
    title= "Gráfico de dispersión 06/11/2020",
    width=500,
    height=350
)
abril13 | nov06

alt.HConcatChart(...)

**Comentarios:** Me parece que si es útil que los tamaños de los círculos sean proporcionales a su población porque así podemos identificar la tendencia de que las comunas más pobladas tengan mayor cantidad de casos confirmados.

## Ejercicio 7

(10 puntos)

1. Grafica la evolución de los casos activos de cada comuna en un solo gráfico. 
2. Grafica la evolución de los casos activos de cada comuna en gráficos separados por región.

Entrega los pros y contras de cada uno de estos enfoques.

In [33]:
alt.Chart(activos.groupby(['comuna','fecha']).agg({'casos_activos':'sum'}).reset_index()).mark_line().encode(
        x=alt.X('fecha' , axis=alt.Axis(title='Fecha')), 
        y=alt.Y('casos_activos' , axis=alt.Axis(title='Casos activos')),
        color= 'comuna'
).properties(
    title= "Evolución de casos activos por todas las comunas de Chile",
    width=1000,
    height=550
)

alt.Chart(...)

In [34]:
activosxregion = []
graf_activosxregion = []
for reg in activos["region"].unique():
    activosxregion.append(activos[activos.region == reg].groupby(['region','comuna','fecha']).agg({'casos_activos':'sum'}).reset_index())

for dfreg in activosxregion:
    c=0
    graf_activosxregion.append(alt.Chart(dfreg).mark_line().encode(
        x=alt.X('fecha' , axis=alt.Axis(title='Fecha')),  
        y=alt.Y('casos_activos' , axis=alt.Axis(title='Casos activos')),
        color= 'comuna'
    ).properties(
    title="Evolución de casos activos por comuna en la región de" + " " + dfreg["region"][0],
    width=500,
    height=280
    ))
    
(graf_activosxregion[0] & graf_activosxregion[1] & graf_activosxregion[2] & graf_activosxregion[3] & graf_activosxregion[4] & graf_activosxregion[5] & graf_activosxregion[6] 
& graf_activosxregion[7] | graf_activosxregion[8] & graf_activosxregion[9] & graf_activosxregion[10] & graf_activosxregion[11] & graf_activosxregion[12]
& graf_activosxregion[13] & graf_activosxregion[14] & graf_activosxregion[15])

alt.HConcatChart(...)

**Comentarios:** 

Un gráfico:
1. Toda la información junta en un solo gráfico. (Pro)
2. Al haber tanta información en un solo grafico esta se solapa haciendo que sea poco legible. (Contra)

Un gráfico por región:

1. Gráficos más claros al tener menos datos por gráfico. (Pro)
2. Se puede perder relaciones que serian encontradas al ver un gráfico con todos los gráficos conjuntos. (Contra)
3. Útil cuando solo se requiere información de alguna región en específico.(Contra)

## Ejercicio 8

(10 puntos)

Hacer un gráfico que permita comparar rápidamente entre regiones su promedio de casos activos , máximo de casos confirmados y fallecidos. Utiliza los valores reales y apoyarlos con colores.

Se adjunta el diccionario `region_names` con tal de reemplazar los nombres de las regiones en los datos `fallecidos` para poder unir con los otros datos. 

In [35]:
region_names = {
    "Araucanía": "La Araucanía",
    "Aysén": "Aysén del General Carlos Ibáñez del Campo",
    "Magallanes": "Magallanes y de la Antártica Chilena",
    "Metropolitana": "Metropolitana de Santiago",
    "O’Higgins": "Libertador General Bernardo O'Higgins",
    "Tarapaca" : "Tarapacá", "Valparaiso" : "Valparaíso" , 
    "Metropolitana" : "Metropolitana de Santiago", 
    "Del Libertador General Bernardo O’Higgins" : "Libertador General Bernardo O'Higgins" ,
    "Biobio" : "Biobío", "La Araucania": "La Araucanía" , "Los Rios" : "Los Ríos" ,
    "Aysen" :'Aysén del General Carlos Ibáñez del Campo', 
    "Magallanes y la Antartica" : "Magallanes y de la Antártica Chilena", "Nuble" : "Ñuble"}  #EL df de confirmados tiene un problema similar (tildes) así que adapte este mismo diccionario para ser usado en ambos df.

fallecidosN = fallecidos
fallecidosN["region"].replace(region_names , inplace=True)  #Nuevo df con los cambios de nombre de región
activosN= activos
activosN["region"].replace(region_names , inplace=True)  #Nuevo df con los cambios de nombre de región

In [36]:
confirmados_region_max=confirmados.groupby(["region","fecha"]).agg(casos_confirmados_max=("casos_confirmados", "max")).reset_index() #df
activos_region=activosN.groupby(["region","fecha"]).agg(casos_activos=("casos_activos", "sum"), casos_activos_mean=("casos_activos", "mean")).reset_index() #df
merge_ddos= pd.merge(confirmados_region_max, activos_region, how="inner", on= ["region","fecha"])  #merge entre confirmados y activos
merge_ttotal= pd.merge(merge_ddos , fallecidos , how="inner", on=["region","fecha"])  #merge total

fecha= merge_ttotal['fecha'].max()  #se puede modificar esta fecha para que el gráfico muestre otro dia

merge_total=merge_ttotal.drop(merge_ttotal[merge_ttotal['fecha'] != fecha].index)  #df solo con el ultimo dia registrado


dia= str(fecha)  

#barra horizontal con la media de las regiones
mediaT = alt.Chart(merge_total).mark_rule(color='#09B309').encode(  
    y='mean(casos_activos)'
    )

#tick con la media de casos activos entre las comunas para cada region
tick = alt.Chart(merge_total).mark_tick(
    color='white',
    thickness=3,
    size=40 * 0.9,  # controls width of tick.
).encode(
    x='region',
    y='casos_activos_mean'
)
#No estaba seguro cual de las dos cosas estaba pidiendo así que implemente ambas (aunque para el tick los valores son cercanos a 0
#así que no se identifican bien en el gráfico)

#grafico de barras de casos_confirmados_max (azul)
graf_confirmados_max=alt.Chart(merge_total).mark_bar(color="#2222bb",opacity=0.45).encode(
    x=alt.X('region' , axis=alt.Axis(title='Region')),
    y=alt.Y('casos_confirmados_max' , axis=alt.Axis(title='Personas')),
    #column='fecha'
  ).properties(
   title="Comparación entre máximo de personas con covid confirmado(Azul),personas fallecidas(Rojo) con linea horizontal en la media de las regiones de Chile para el dia:" + " " + dia ,
    width=950,
    height=500
    )
#grafico de barras de fallecidos (rojo)
graf_fallecidos=alt.Chart(merge_total).mark_bar(color="red",opacity=0.9).encode(
    x=alt.X('region' , axis=alt.Axis(title='Region')),
    y=alt.Y('fallecidos' , axis=alt.Axis(title='Personas')),
    #column='fecha'
   ).properties(
    title="Comparación entre personas con covid confirmado(Azul),personas fallecidas(Rojo) con linea horizontal en la media de las regiones de Chile para el dia:" + " " + dia ,
    width=950,
    height=500
    )
mediaT+ graf_confirmados_max +graf_fallecidos + tick 


alt.LayerChart(...)

**Comentarios:** -

## Ejercicio 9


En este ejercicio buscaremos realizar un mini-dashboard respecto al estado de los casos de COVID-19 en Chile, por lo tanto utilizaremos haremos uso de datos geográficos de manera operacional (es decir, no nos preocuparemos de proyecciones en mapas ni nada por el estilo), lo único es que debes instalar `geopandas` en tu ambiente virtual y no olvidar actualizarlo en tu `environment.yml` para luego subirlo a tu repositorio de GitHub.

Con tu ambiente activo (`conda activate mat281`) basta con ejecutar `conda install -c conda-forge geopandas` para instalar `geopandas`.

In [37]:
import geopandas as gpd
from pathlib import Path

In [38]:
shp_filepath = Path().resolve().parent / "data" / "regiones_chile.shp"
regiones = gpd.read_file(shp_filepath)
regiones

,Region,objectid,cir_sena,codregion,area_km,st_area_sh,st_length_,geometry
0,Región Metropolitana de Santiago,1092,7,13,15392.030737,2.225204e+10,1.064253e+06,"POLYGON ((-7873736.745 -3885505.642, -7873695...."
1,Región de Antofagasta,1086,3,2,126071.431981,1.508452e+11,2.516112e+06,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
2,Región de Arica y Parinacota,1084,1,15,16866.819844,1.886869e+10,7.505296e+05,"POLYGON ((-7727277.278 -1997230.768, -7726464...."
3,Región de Atacama,1089,4,3,75661.248635,9.643906e+10,2.401741e+06,"MULTIPOLYGON (((-7900342.628 -3153340.296, -79..."
4,Región de Aysén del Gral.Ibañez del Campo,1088,14,11,106703.377369,2.242743e+11,4.144481e+07,"MULTIPOLYGON (((-8208500.834 -5733817.475, -82..."
5,Región de Coquimbo,1090,5,4,40575.901682,5.498082e+10,2.065933e+06,"MULTIPOLYGON (((-7977004.630 -3643350.078, -79..."
6,Región de La Araucanía,1095,11,9,31837.567634,5.221507e+10,1.501025e+06,"POLYGON ((-8101075.126 -4521676.016, -8101101...."
7,Región de Los Lagos,1093,13,10,48408.371357,8.771834e+10,7.874158e+06,"MULTIPOLYGON (((-8224346.368 -5128442.053, -82..."
8,Región de Los Ríos,1094,12,14,18245.080563,3.108661e+10,1.844423e+06,"MULTIPOLYGON (((-8160194.639 -4859056.006, -81..."
9,Región de Magallanes y Antártica Chilena,1087,15,12,133053.142345,3.581316e+11,9.049830e+07,"MULTIPOLYGON (((-8024802.991 -7248340.305, -80..."


In [19]:
type(regiones)

geopandas.geodataframe.GeoDataFrame

Lo único que tienes que saber es que un `GeoDataFrame` es idéntico a un `DataFrame` salvo que debe poseer una columna llamada `geometry` caracterice los elementros geométricos, que en este casos son polígonos con los límites de las regiones de Chile. 

Para graficar mapas en Altair se debe usar `mark_geoshape`, además, para no preocuparnos de las proyecciones si o si debes declarar lo siguiente que se muestra en la siguiente celda en las propiedades del gráfico. El resto es igual a cualquier otro gráfico de Altair.

In [20]:
alt.Chart(regiones).mark_geoshape().encode(
    color= 'codregion'
).properties(
    projection={'type': 'identity', 'reflectY': True},
    width=250,
    height=600
)

alt.Chart(...)

### Ejercicio 9.1

(10 puntos)

Define el `DataFrame` con el nombre `casos_geo` tal que tenga las columnas

* `region`
* `codigo_region`
* `fecha`
* `poblacion`
* `casos_confirmados`
* `tasa`
* `casos_activos`
* `fallecidos`
* `geometry`

Ten mucho cuidado como unes los dataframes `confirmados`, `activos`, `fallecidos` y `regiones`. Idealmente utilizar el código de región, pero en caso que no se encuentren disponibles utilizar el nombre de la región (no olivdar utilizar el diccionario `region_names`).

In [39]:
confirmadosfil=confirmados.groupby(["fecha","region"]).agg(    
        poblacion=("poblacion", "sum"),
        casos_confirmados=("casos_confirmados", "sum"),
        tasa=("tasa","sum")
    ).reset_index()  #df confirmados editado de forma conveniente

activosfil=activosN.groupby(["region","fecha","codigo_region"]).agg({"casos_activos":"sum"}).reset_index() #df activos editado de forma conveniente

regiones_fil=regiones.drop(["Region","objectid","cir_sena","area_km","st_area_sh","st_length_"],axis=1).rename(columns={"codregion":"codigo_region"}) #df regiones editado de forma conveniente

conf_y_act_fil=pd.merge(confirmadosfil,activosfil, how= 'inner' ,on=["region","fecha"])  #merge entre confirmadosfill y activosfill
conf_y_act_fall_fil=pd.merge(conf_y_act_fil, fallecidosN, how='inner', on=["region","fecha"]) #merge entre el df creado en la linea previa y fallecidosN

In [40]:
casos_geo = (
    pd.merge(conf_y_act_fall_fil,regiones_fil, how='inner',on="codigo_region" )
)

casos_geo.head()

,fecha,region,poblacion,casos_confirmados,tasa,codigo_region,casos_activos,fallecidos,geometry
0,2020-04-13,Antofagasta,691854.0,149.0,202.281988,2,78.0,1.0,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
1,2020-04-15,Antofagasta,691854.0,176.0,274.749950,2,88.0,1.0,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
2,2020-04-17,Antofagasta,691854.0,211.0,361.149394,2,103.0,1.0,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
3,2020-04-20,Antofagasta,691854.0,264.0,455.904971,2,129.0,1.0,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
4,2020-04-24,Antofagasta,691854.0,368.0,602.205002,2,172.0,2.0,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."


Ejecuta lo siguiente para convertir el DataFrame anterior en un GeoDataFrames

In [41]:
casos_geo = casos_geo.pipe(lambda x: gpd.GeoDataFrame(x, geometry="geometry"))

### Ejercicio 9.2

(5 puntos)

Modifica la función `covid_chile_chart` tal que reciba una fecha y una columna. Luego, debe filtrar `casos_geo` con registros de la fecha seleccionada y graficar un mapa donde las regiones se colereen según la columna escogida. 

In [42]:
def covid_chile_chart(fecha, col):
    
    fecha = pd.to_datetime(fecha)
    data = casos_geo.drop(casos_geo[casos_geo['fecha'] != fecha].index)
    
    chart = alt.Chart(data).mark_geoshape().encode(
    color= col
    ).properties(
        projection={'type': 'identity', 'reflectY': True},
        width=150,
        height=400
    )
    
    chart.display()
    return

Prueba con lo siguiente

In [43]:
fecha = "2020-04-13"  
col = "tasa"
covid_chile_chart(fecha, col) #Se demora bastante  #No mostro el mapa en el binder

alt.Chart(...)

### Ejercicio 9.3

(5 puntos)

Ahora utilizando `widgets` generaremos el dashboard interactivo. Define lo siguiente:

* col_widget: Un `widgets.Dropdown` donde las opciones a seleccionar sean las columnas `poblacion`, `casos_confirmados`, `tasa`, `casos_activos` y `fallecidos`. Además, el argumento `description` debe ser `Columna`.
* fecha_widget: Un `widgets.DatePicker` donde el argumento `description` sea `Fecha`.
* Ambos widgets deben tener el argumento `continuous_update=False`

In [44]:
import ipywidgets as widgets
from ipywidgets import interactive, interact

In [45]:
col_widget = widgets.Dropdown(options=["poblacion","casos_confirmados","tasa","casos_activos","fallecidos"], description="Columna")

In [46]:
fecha_widget = (widgets.DatePicker(
    description="Fecha",
))

Finalmente, haciendo uso de `interactive`, la función `covid_chile_chart` y todos los widgets es posible crear un _dashboard_ interactivo con los datos de Covid-19.

Respira profundo y explora tu creación!

In [47]:
covid_dashboard = interactive(
    covid_chile_chart,
    fecha=fecha_widget,
    col=col_widget
)
covid_dashboard

alt.Chart(...)

**Comentarios:** Los mapas no se mostraban en binder, pero en mi computador si (excepto interactive que no era compatible con windowss 7)